In [1]:
import os
import csv
import datetime
import pandas as pd
from tqdm import tqdm

KeyboardInterrupt: 

# first extract the data

In [ ]:
folder_list = []
filelist = os.listdir('E:\OneDrive\TLEs\origin\\2004_pre\\')
for item in filelist:
    folder_list.append('E:\OneDrive\TLEs\origin\\2004_pre\\'+item)

for folder in folder_list:
    files = os.listdir(folder)
    for item in tqdm(files):
        if item.endswith('.csv'):
            # item = files[0]
            try:
                path = f'{folder}\\{item}'
                name = item[0:-12]
                part = item[-6:-4]
                df_temp = pd.read_csv(path,index_col=0)

                df_temp.insert(1,'year',0)
                for time in df_temp.index:
                    y = datetime.datetime.fromtimestamp(time).year
                    df_temp['year'].at[time] = y

                year_list = df_temp['year'].drop_duplicates()

                for y in year_list:
                    df_year = df_temp[df_temp['year'] == y]
                    df_year = df_year.drop(columns=['year'])
                    if y>2004:
                        part = y-2030
                    if y>2022:
                        y=y-100
                    if not os.path.exists(f'E:\\OneDrive\\TLEs\\tle{y}.txt'):
                        os.makedirs(f'E:\\OneDrive\\TLEs\\tle{y}.txt')
                    storage_path = f'E:\\OneDrive\\TLEs\\tle{y}.txt\\{name}_{y}_{part}.csv'
                    df_year.to_csv(storage_path)
            except:
                pass
        

100%|██████████| 26936/26936 [23:17<00:00, 19.27it/s] 


# second combine the data

In [ ]:
folders = os.listdir('E:\\OneDrive\\TLEs')
folders.remove('final_mat_files')
folders.remove('origin')
folder_to_delete = []
for folder in folders:
    if (folder[0:3] == 'new') or int(folder[3:7]) > 2023:
        folder_to_delete.append(folder)

for folder in folder_to_delete:
    folders.remove(folder)

for folder in folders:
    yy = int(folder[3:7])
    id_list = []
    part_list = []
    files = os.listdir(f'E:\\OneDrive\\TLEs\\{folder}')
    for file in files:
        part_list.append(file[-6:-4])
        id_list.append(file[0:-12])
    df = pd.DataFrame({'file':files,'id':id_list,'part':part_list})
    id_dedup = df['id'].drop_duplicates()
    for id in tqdm(id_dedup):
        df_temp = df[df['id'] == id]
        data_combine = []
        for i,f in enumerate(df_temp['file']):
            path = f'E:\\OneDrive\\TLEs\\{folder}\\{f}'
            data_one_file = open(path).readlines()
            data_combine = data_combine+data_one_file[1:]

        if not os.path.exists(f'E:\\OneDrive\\TLEs\\new_{folder}'):
            os.makedirs(f'E:\\OneDrive\\TLEs\\new_{folder}')
        f2 = open(f'E:\\OneDrive\\TLEs\\new_{folder}\\{id}_{folder[3:7]}.csv','w',newline='')
        writer = csv.writer(f2)
        header = ['time','inclination','eccentricity','AoP','mean_anomaly','mean_motion']
        writer.writerow(header)
        f2.writelines(data_combine)
        f2.close()

100%|██████████| 26885/26885 [05:20<00:00, 83.79it/s] 
